Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [37]:
from random import choices, randint
import numpy as np
import copy

import lab9_lib

In [ ]:
fitness = lab9_lib.make_problem(1)
for n in range(10):     #Determina il numero di Fitness call
    ind = choices([0, 1], k=1000)     #genoma casuale di 50 elementi
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

# Local Search Algorithm

In [34]:
# GLOBAL PARAMETER #
NUM_LOCI = 1000

POPULATION = 10
NEW_OFFSPRING = 30
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .15

# . . . . . . #

### Mutation Techniques

In [ ]:
def mutation1(ind: fitness) -> fitness:
    offspring = copy(ind)
    pos = randint(0, NUM_LOCI-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def mutation2(ind: fitness) -> fitness:
    offspring = copy(ind)
    pos = randint(0, NUM_LOCI-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def mutation3(ind: fitness) -> fitness:
    offspring = copy(ind)
    pos = randint(0, NUM_LOCI-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def mutation4(ind: fitness) -> fitness:
    offspring = copy(ind)
    pos = randint(0, NUM_LOCI-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

### Recombination Techniques

In [ ]:
def one_cut_crossover(ind1: fitness, ind2: fitness) -> fitness:
    cut_point = randint(0, NUM_LOCI-1)
    #offspring = (fitness = None,
    #                       genotype= ind1.genotype[:cut_point]+ind2.genotype[cut_point:])
    assert len(offspring.genotype) == NUM_LOCI
    return offspring



### Survival Selection

### Island Definition

## Problem start from HERE ->

In [48]:
starting_population = [choices([0, 1], k=NUM_LOCI) for _ in range(POPULATION)]

In [ ]:
[print(starting_population[i]) for i in range(len(starting_population))]